> DUPLICATE THIS COLAB TO START WORKING ON IT. Using File > Save a copy to drive.


### Word2Vec

In this notebook we're going to learn and play around with word2vec embeddings that are packaged with Spacy. We'll try to build intuition on how they work and what can we do with them.

Install all the required dependencies for the project

In [36]:
%%capture
!pip install spacy==2.2.4 --quiet
!python -m spacy download en_core_web_md
!apt install libopenblas-base libomp-dev
!pip install faiss-cpu

Import all the necessary libaries

In [37]:
from collections import defaultdict
import en_core_web_md
import numpy as np
import spacy
import time
import faiss

Now let's load the Spacy data which comes with pre-trained embeddings. This process is expensive so only do this once.



In [38]:
spacyModel = en_core_web_md.load()

First, let's play with some basic similarity functions.

In [44]:
# API can be found here: https://spacy.io/api/doc
banana = spacyModel("girl")
fruit = spacyModel("women")
table = spacyModel("table")
print(banana.similarity(fruit))
print(banana.similarity(table))



0.5421239860059744
0.2619973944685995


As expected `Banana` is a lot more similar to `Fruit` than to `Table`. Now let's iterate over the entire vocabulary and build a search index using **Faiss**. This will make it a lot faster for us to find similar words instead of having to loop over the entire corpus each time. 

Feel free to ignore learning more about **Faiss** at this time as we'll dive more into it in Week 3. At the high-level it is a really efficient library to find similar vectors from a corpus.

Note: This next cell will take a fair bit of time to run.

In [45]:
def load_index():
  """Expensive method - call only once!!
  """
  word_to_id = {}
  id_to_word = {}
  vectors = []
  vector_dimension = 300
  id = 0

  # Iterate over the entire vocabulary
  for i, tok in enumerate(spacyModel.vocab):
    vector = tok.vector
    l2_norm = np.linalg.norm(vector)

    # Ignore zero vectors, nan vlaues
    if (np.isclose(l2_norm, 0.0) or 
        np.isnan(l2_norm) or 
        np.any(np.isnan(vector))):
      continue
    else:
      vectors.append(np.divide(vector, l2_norm))

    # Add to the output variables
    word_to_id[tok.text.lower()] = id
    id_to_word[id] = tok.text.lower()
    id += 1

  
  vectors = np.array(vectors)
  index = faiss.IndexFlatIP(vector_dimension)
  index.add(vectors)
  return word_to_id, id_to_word, vectors, index

word_to_id, id_to_word, vectors, index = load_index()
vector_size = len(vectors)
print("We created a search index of %d vectors" % vector_size)

We created a search index of 684754 vectors


Now we're going to add a helper functions to calculate top_k similar words to some input in the index.

In [46]:
def search_vector(word_vector, top_k=100, print_time_taken=False):
  word_vector = np.array([word_vector])
  start_time = time.time()
  scores, indices = index.search(word_vector, top_k)
  if print_time_taken:
    print("Time taken to search amongst {} words is {:.3}s".format(
        vector_size, (time.time() - start_time))
    )
  results = []
  words = set()
  for i, query_index in enumerate(indices):
      # Matches for the i'th one 
      for inn_idx, word_index in enumerate(query_index):
          if word_index < 0:
              continue
          word = id_to_word[word_index]
          if word in words:
            continue
          words.add(word)
          results.append((word, float(scores[i][inn_idx])))
  return sorted(results, key=lambda tup: -tup[1])

Let's do an empirical test by searching similar words to a few terms

### Search

In [47]:
def search(word, top_k=100,print_time_taken=False):
  word = word.lower()
  if word not in word_to_id:
    print("Oops, the word {} is not in loaded dictionary".format(word))
    return
  id = word_to_id[word]
  word_vector = vectors[id]
  search_results = search_vector(word_vector, top_k, print_time_taken)
  print(f"The top similar words to {word} are - ")
  for i in range(len(search_results)):
    print(f"Word = {search_results[i][0]} and similarity = {search_results[i][1]}")
  return search_results

In [66]:
output = search("portland", 100, True)

Time taken to search amongst 684754 words is 0.0355s
The top similar words to portland are - 
Word = beaverton and similarity = 1.0
Word = pdx and similarity = 1.0
Word = pdxpipeline and similarity = 1.0
Word = sellwood and similarity = 1.0
Word = portland and similarity = 1.0
Word = sammamish and similarity = 1.0
Word = milwaukie and similarity = 1.0
Word = seatac and similarity = 0.8688419461250305
Word = redmond and similarity = 0.8688419461250305
Word = belltown and similarity = 0.8688419461250305
Word = safeco and similarity = 0.8688419461250305
Word = tukwila and similarity = 0.8688419461250305
Word = lynnwood and similarity = 0.8688419461250305
Word = showbox and similarity = 0.8688419461250305
Word = bothell and similarity = 0.8688419461250305
Word = renton and similarity = 0.8688419461250305
Word = mukilteo and similarity = 0.8688419461250305
Word = bellevue and similarity = 0.8688419461250305
Word = seattle and similarity = 0.8688419461250305
Word = poulsbo and similarity = 0

In [ ]:
output = search("baseball", 10)

In [ ]:
output = search("cheese", 25)

Now why don't you try out a few different words that come to mind and see where does the model perform well and where it struggles!! 

### Analogies

In [67]:
def analogy(word1, word2, word3):
  word1 = word1.lower()
  word2 = word2.lower()
  word3 = word3.lower()
  if word1 not in word_to_id or word2 not in word_to_id or word3 not in word_to_id:
    print("word not present in dictionary, try something else")
  vector1 = vectors[word_to_id[word1]]
  vector2 = vectors[word_to_id[word2]]
  vector3 = vectors[word_to_id[word3]]
  analogy_results = search_vector(np.add(np.subtract(vector1, vector2), vector3), 10)
  print(f"The top similar item for ({word1} - {word2} + {word3}) = {analogy_results[0][0]}")
  print(f"The top similar words to ({word1} - {word2} + {word3}) are - ")
  for i in range(len(analogy_results)):
    print(f"Word = {analogy_results[i][0]} and similarity = {analogy_results[i][1]}")
  return analogy_results

In [76]:
output = analogy("test", "test", "test")

The top similar item for (test - test + test) = test
The top similar words to (test - test + test) are - 
Word = test and similarity = 0.9999998807907104
Word = retest and similarity = 0.9063977003097534
Word = polygraph and similarity = 0.9063977003097534
Word = pre-test and similarity = 0.9063977003097534
Word = tests and similarity = 0.9063977003097534
Word = lie-detector and similarity = 0.9063977003097534


In [78]:
output = analogy("smallest", "small", "short")

The top similar item for (smallest - small + short) = shortest
The top similar words to (smallest - small + short) are - 
Word = shortest and similarity = 0.8045328259468079
Word = straightest and similarity = 0.8045328259468079
Word = sixth-longest and similarity = 0.7076637744903564
Word = longest-running and similarity = 0.7076637744903564
Word = third-longest and similarity = 0.7076637744903564
Word = longest-ever and similarity = 0.7076637744903564


Now why don't you try out a few different examples see what comes out :) 